In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
tf.device('cuda:0')

2022-10-20 18:46:00.762923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 18:46:00.856153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 18:46:00.856920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 18:46:00.866355: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
# train_df=pd.read_excel('../input/ire-major-project-data/train.xlsx')
# dev_df=pd.read_excel('../input/ire-major-project-data/dev.xlsx')
# test_df=pd.read_excel('../input/ire-major-project-data/test.xlsx')

train_df=pd.read_table('../input/ire-major-data-v2/en_train_pt.tsv',names=['SNO','Intensity','Text'],sep='\t')
test_df=pd.read_table('../input/ire-major-data-v2/en_test_pt.tsv',names=['SNO','Intensity','Text'],sep='\t')

In [5]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [6]:
def return_length(df):
    length=[]
    for i in df['Text']:
        tok = tokenizer(i, return_tensors="pt")
        length.append(len(tok['input_ids'][0]))
        
    return length

In [7]:
train_df['#tokens']=return_length(train_df)
test_df['#tokens']=return_length(test_df)

Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors


In [8]:
train_df=train_df[train_df['#tokens']<50]
test_df=test_df[test_df['#tokens']<50]
print('Length of Train_df:',len(train_df))
print('Length of Test_df:',len(test_df))

Length of Train_df: 17497
Length of Test_df: 4337


In [9]:
train_df['label']=['clickbait' if i>=0.5 else 'no-clickbait' for i in train_df['Intensity']]
test_df['label']=['clickbait' if i>=0.5 else 'no-clickbait' for i in test_df['Intensity']]

## Replacing/Masking Words
- Replace Non Named Entities
- Replace Pronouns
- Replace Random Words

In [10]:
# ! pip install spacy --q
# ! pip install nltk --q
! python -m spacy download en_core_web_sm --q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
import spacy
from spacy import displacy
from spacy import tokenizer
nlp = spacy.load('en_core_web_sm')

In [12]:
def get_entities(text):
    doc = nlp(text)
    tokens=[token.text for token in doc]
    ent_tokens=[]
    ent_labels=[]
    for e in doc.ents:
        toks=e.text.split()
        for tok in toks:
            ent_tokens.append(tok)
            ent_labels.append(e.label_)
#     displacy.render(doc, style='ent', jupyter=True)
    #return tokens,ent_tokens,ent_labels
    
    #print(tokens,'\n',ent_tokens,'\n',ent_labels)
    return tokens,ent_tokens,ent_labels

In [13]:
def get_pos_tags(text):
    sent = text
    doc=nlp(sent)

    sub_toks = [tok for tok in doc if ((tok.dep_ == "nsubj") )]
    nc= [x for x in doc.noun_chunks]
    l=[]
    for i,token in enumerate(doc):
        #print(token.pos_)
        if token.pos_ in ['PRON','DET']:
            #l.append([token.text,i,token.pos_])
            l.append(token.text)

    #print(l)
    return l

In [14]:
new_test_sents=[]
for i in test_df['Text']:
    tokens,ent_tokens,ent_labels=get_entities(i)
    pos_tag_tokens=get_pos_tags(i)
    new_tokens=[]
    for token in tokens:
        if ((token not in ent_tokens) & (token in pos_tag_tokens)):
            continue
        else:
            new_tokens.append(token)
    new_text=' '.join(new_tokens).strip()
    new_test_sents.append(new_text)

In [15]:
from transformers import RobertaTokenizer, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model = RobertaModel.from_pretrained("roberta-large")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
model.to('cuda:0')

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (d

In [17]:
def read_word_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt").to('cuda:0')
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state.cpu()
    embedding=np.mean(last_hidden_states[0].detach().numpy(),axis=0)
#     embedding=np.mean(last_hidden_states[0].detach().to('cpu'),axis=0)
    return embedding

In [18]:
x_train_embeddings=[]
for i in tqdm(range(len(train_df))):
    try:
        x_train_embeddings.append(read_word_embedding(train_df['Text'].iloc[i]))
    except:
        print(i)
        break

100%|██████████| 17497/17497 [05:36<00:00, 51.98it/s]


In [19]:
x_train_embeddings=np.array(x_train_embeddings)
y_train_intensity=np.array(list(train_df['Intensity'].values))

In [20]:
# x_dev_embeddings=[read_word_embedding(sentence) for sentence in tqdm(dev_df['text'])]
# y_dev_intensity=list(dev_df['intensity'].values)
# y_dev_actual_label=list(dev_df['class'].values)

In [21]:
x_test_embeddings=[]
for i in tqdm(range(len(test_df))):
    try:
        x_test_embeddings.append(read_word_embedding(test_df['Text'].iloc[i]))
    except:
        print(i)
        break

100%|██████████| 4337/4337 [01:21<00:00, 53.53it/s]


In [22]:
x_test_embeddings=np.array(x_test_embeddings)
y_test_intensity=np.array(list(test_df['Intensity'].values))
y_test_actual_label=np.array(list(test_df['label'].values))

In [23]:
x_test_embeddings_new=[]
for i in tqdm(new_test_sents):
    try:
        x_test_embeddings_new.append(read_word_embedding(i))
    except:
        print(i)
        break

100%|██████████| 4337/4337 [01:20<00:00, 53.60it/s]


# Linear Regression

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [25]:
linear_reg = LinearRegression().fit(x_train_embeddings, y_train_intensity)

In [26]:
y_test_pred = linear_reg.predict(x_test_embeddings)
y_test_new_pred = linear_reg.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [27]:
actual_sents=list(test_df['Text'].values)

In [28]:
out_df=pd.DataFrame(list(zip(actual_sents,y_test_intensity,y_test_pred,new_test_sents,y_test_new_pred)),columns=['actual_sentence','actual_intensity','predicted_intensity','modified_sentence','modified_predicted_intensity'])

In [29]:
# out_df.head()

In [30]:
out_df.to_csv('./Test_Results_LR.csv')

In [31]:
from sklearn.metrics import median_absolute_error,mean_squared_error, f1_score, accuracy_score

In [32]:
def results(y_true,y_pred,y_actual_classes,y_pred_classes):
    Medae = median_absolute_error(y_true,y_pred)
    mse = mean_squared_error(y_true,y_pred) #default=True
    rmse = mean_squared_error(y_true,y_pred,squared=False)
    accuracy=accuracy_score(y_actual_classes,y_pred_classes)
    f1=f1_score(y_actual_classes,y_pred_classes,pos_label='clickbait')
    
    print("MedAE:",Medae)
    print("MSE:",mse)
    print("RMSE:",rmse)
    print("Accuracy:",accuracy)
    print("f1-score:",f1)

In [33]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.10460656484634766
MSE: 0.027230679171916145
RMSE: 0.16501720871447362
Accuracy: 0.8605026516024902
f1-score: 0.6886258363355636


In [34]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.11332241027465823
MSE: 0.02963505966311678
RMSE: 0.17214836526414296
Accuracy: 0.8480516486050265
f1-score: 0.6615305598356446


# Ridge Regression

In [35]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=2).fit(x_train_embeddings, y_train_intensity)

In [36]:
y_test_pred = clf.predict(x_test_embeddings)
y_test_new_pred = clf.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [37]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.1036982536315918
MSE: 0.026949155153049673
RMSE: 0.16416197840258162
Accuracy: 0.8607332257320729
f1-score: 0.6870466321243524


In [38]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.11304701835998535
MSE: 0.029345788202809225
RMSE: 0.17130612424198158
Accuracy: 0.8478210744754439
f1-score: 0.6551724137931034


# Gradient Boosted Regression

In [39]:
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor().fit(x_train_embeddings, y_train_intensity)

In [40]:
y_test_pred = clf.predict(x_test_embeddings)
y_test_new_pred = clf.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [41]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.11595803654278576
MSE: 0.029872405529428474
RMSE: 0.17283635476782214
Accuracy: 0.8434401660133733
f1-score: 0.6291643910431457


In [42]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.11723916595117786
MSE: 0.03231203256546582
RMSE: 0.17975547993167224
Accuracy: 0.8321420336638229
f1-score: 0.5863636363636364


# Random Forest Regression

In [43]:
from sklearn.ensemble import RandomForestRegressor
clf= RandomForestRegressor(max_depth=3, random_state=0).fit(x_train_embeddings, y_train_intensity)

In [44]:
y_test_pred = clf.predict(x_test_embeddings)
y_test_new_pred = clf.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [45]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.13800783686471818
MSE: 0.04126585778480399
RMSE: 0.2031399955321551
Accuracy: 0.8086234724463915
f1-score: 0.5018007202881153


In [46]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.13645321857952353
MSE: 0.042124003601247434
RMSE: 0.20524133014879686
Accuracy: 0.8109292137422182
f1-score: 0.5265588914549654


# Adaboost Regression

In [47]:
from sklearn.ensemble import AdaBoostRegressor
clf = AdaBoostRegressor(random_state=0, n_estimators=100).fit(x_train_embeddings, y_train_intensity)

In [48]:
y_test_pred = clf.predict(x_test_embeddings)
y_test_new_pred = clf.predict(x_test_embeddings_new)
y_test_pred_classes=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_pred]
y_test_pred_classes_new=['clickbait' if i>=0.5 else 'no-clickbait' for i in y_test_new_pred]
y_test_actual_classes=list(test_df['label'].values)

In [49]:
print('Without Masking/Replacing Words\n')
results(y_test_intensity,y_test_pred,y_test_actual_classes,y_test_pred_classes)

Without Masking/Replacing Words

MedAE: 0.16543360946503513
MSE: 0.04043156855426112
RMSE: 0.20107602680145914
Accuracy: 0.8360617938667282
f1-score: 0.595793064241046


In [50]:
print('With Masking/Replacing Words\n')
results(y_test_intensity,y_test_new_pred,y_test_actual_classes,y_test_pred_classes_new)

With Masking/Replacing Words

MedAE: 0.16634053668375837
MSE: 0.0418010933607677
RMSE: 0.2044531568862846
Accuracy: 0.8185381600184459
f1-score: 0.5120892746435214
